# Lista de Exercícios #10: RNA - Perceptron e Backpropagation

**Aluno:** Samuel Horta de Faria
**Matrícula:** 801528

Este notebook contém as implementações e análises para os algoritmos Perceptron e Backpropagation.

## Configurações Iniciais e Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import time

# Para o Exercício 2 (Backpropagation)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split # Pode ser útil para bases maiores

# Exercício 1: Perceptron

### 1.1 Geração de Dados para Funções Lógicas (n entradas)

In [ ]:
def gerar_dados_logicos(n_entradas, operacao):
    """Gera todas as combinacoes de entradas booleanas e suas respectivas saidas para AND, OR, XOR."""
    entradas = list(product([0, 1], repeat=n_entradas))
    X = np.array(entradas)
    y = []
    if operacao == 'AND':
        y = np.all(X, axis=1).astype(int)
    elif operacao == 'OR':
        y = np.any(X, axis=1).astype(int)
    elif operacao == 'XOR':
        if n_entradas == 1:
            y = X[:,0].astype(int) # XOR de 1 entrada é a própria entrada
        elif n_entradas == 2: # XOR tradicional
             y = np.logical_xor(X[:,0], X[:,1]).astype(int)
        else: # XOR generalizado (paridade: 1 se número ímpar de 1s)
            y = (np.sum(X, axis=1) % 2 != 0).astype(int)
    else:
        raise ValueError("Operação não suportada. Escolha 'AND', 'OR' ou 'XOR'.")
    return X, y.reshape(-1, 1)

### 1.2 Implementação do Perceptron

In [ ]:
class Perceptron:
    def __init__(self, num_inputs, learning_rate=0.1):
        self.learning_rate = learning_rate
        # Inicializa pesos: num_inputs + 1 para o bias
        # Pesos entre -0.5 e 0.5 para evitar grandes passos iniciais
        self.weights = (np.random.rand(num_inputs + 1) - 0.5) * 0.1 
        self.errors_ = []
        self.weights_history_ = [] # Para plotar o hiperplano

    def predict(self, inputs):
        # Adiciona o bias (entrada x0 = 1)
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0] # weights[0] é o bias
        return 1 if summation >= 0 else 0 # Função degrau

    def train(self, training_inputs, labels, epochs=100):
        self.errors_ = []
        self.weights_history_ = [self.weights.copy()]
        
        # Adiciona coluna de 1s para o bias nas entradas se não presente
        # No nosso caso, o bias é tratado internamente no `predict` e atualização
        # Esta implementação considera que `training_inputs` não tem a coluna de bias

        for _ in range(epochs):
            epoch_errors = 0
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                error = label - prediction
                if error != 0:
                    epoch_errors += 1
                    self.weights[1:] += self.learning_rate * error * inputs
                    self.weights[0] += self.learning_rate * error # Atualiza o bias
            self.errors_.append(epoch_errors)
            self.weights_history_.append(self.weights.copy())
            if epoch_errors == 0: # Convergiu
                print(f"Convergiu em {_ + 1} épocas.")
                break
        return self.errors_

### 1.3 Função para Plotar Hiperplano de Separação (para n=2)

In [ ]:
def plotar_hiperplano(X, y, perceptron_model, title="", epoch_to_plot=-1):
    if X.shape[1] != 2:
        print("Plotagem do hiperplano só é suportada para 2 entradas.")
        return

    plt.figure(figsize=(8, 6))
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), cmap='coolwarm', edgecolors='k', s=100, label='Dados')

    # Pega os pesos da época especificada (ou a última)
    if not perceptron_model.weights_history_:
        print("Modelo não treinado ou histórico de pesos não disponível.")
        return
        
    weights_to_plot = perceptron_model.weights_history_[epoch_to_plot]
    w0 = weights_to_plot[0] # Bias
    w1 = weights_to_plot[1]
    w2 = weights_to_plot[2]

    # Linha de decisão: w1*x1 + w2*x2 + w0 = 0
    # x2 = (-w1*x1 - w0) / w2
    x_vals = np.array([np.min(X[:,0]) - 0.5, np.max(X[:,0]) + 0.5])
    if w2 != 0: # Evita divisão por zero se w2 for zero
        y_vals = (-w1 * x_vals - w0) / w2
        plt.plot(x_vals, y_vals, 'k--', label=f'Hiperplano (Época {len(perceptron_model.weights_history_) if epoch_to_plot == -1 else epoch_to_plot})')
    elif w1 != 0: # Se w2 é 0, a linha é vertical: x1 = -w0 / w1
        x_vert = -w0 / w1
        plt.axvline(x=x_vert, color='k', linestyle='--', label=f'Hiperplano (Época {len(perceptron_model.weights_history_) if epoch_to_plot == -1 else epoch_to_plot})')
    else: # Se w1 e w2 são 0, não há linha de decisão clara (ou é tudo uma classe)
        print("Não foi possível plotar o hiperplano (w1 e w2 são zero).")

    plt.xlim(np.min(X[:,0]) - 0.5, np.max(X[:,0]) + 0.5)
    plt.ylim(np.min(X[:,1]) - 0.5, np.max(X[:,1]) + 0.5)
    plt.xlabel("Entrada X1")
    plt.ylabel("Entrada X2")
    plt.title(f"Perceptron: {title}")
    plt.legend()
    plt.grid(True)
    plt.show()

def plotar_erro(errors, title):
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(errors) + 1), errors, marker='o')
    plt.xlabel("Épocas")
    plt.ylabel("Número de Erros de Classificação")
    plt.title(f"Curva de Aprendizado Perceptron: {title}")
    plt.grid(True)
    plt.show()

### 1.4 Testes com Perceptron

In [ ]:
def run_perceptron_experiment(n_entradas, operacao, epochs=100, learning_rate=0.1):
    print(f"\n--- Testando Perceptron para {operacao} com {n_entradas} entradas ---")
    X, y = gerar_dados_logicos(n_entradas, operacao)
    
    # Adicionar coluna de bias (1s) aos dados de entrada X para o Perceptron
    # Nossa implementação atual lida com bias internamente, então X original é usado.
    
    perceptron = Perceptron(num_inputs=n_entradas, learning_rate=learning_rate)
    errors = perceptron.train(X, y.ravel(), epochs=epochs)
    
    print(f"Pesos finais (w0=bias, w1, ...): {perceptron.weights}")
    
    # Testar predições
    print("Predições para todas as entradas:")
    correct_predictions = 0
    for i in range(len(X)):
        pred = perceptron.predict(X[i])
        if pred == y[i][0]:
            correct_predictions +=1
        print(f"Entrada: {X[i]}, Esperado: {y[i][0]}, Predito: {pred}")
    accuracy = correct_predictions / len(X)
    print(f"Acurácia final: {accuracy*100:.2f}%")
    
    title = f"{operacao} com {n_entradas} entradas"
    plotar_erro(errors, title)
    
    if n_entradas == 2:
        plotar_hiperplano(X, y, perceptron, title)
        # Plotar hiperplano no início do treinamento (após 1a época, por exemplo)
        if len(perceptron.weights_history_) > 1:
             plotar_hiperplano(X, y, perceptron, title + " (Após 1ª época com atualizações)", epoch_to_plot=1 if len(errors)>0 and errors[0] > 0 else 0)

# Testes para AND
run_perceptron_experiment(n_entradas=2, operacao='AND', epochs=100)
run_perceptron_experiment(n_entradas=3, operacao='AND', epochs=100)
run_perceptron_experiment(n_entradas=5, operacao='AND', epochs=200, learning_rate=0.05)

# Testes para OR
run_perceptron_experiment(n_entradas=2, operacao='OR', epochs=100)
run_perceptron_experiment(n_entradas=3, operacao='OR', epochs=100)
run_perceptron_experiment(n_entradas=5, operacao='OR', epochs=200, learning_rate=0.05)

# Teste para XOR (demonstrar que não resolve)
run_perceptron_experiment(n_entradas=2, operacao='XOR', epochs=100)
run_perceptron_experiment(n_entradas=3, operacao='XOR', epochs=300, learning_rate=0.05) # XOR generalizado

### 1.5 Análise dos Resultados do Perceptron

**Funções AND e OR:**
O Perceptron conseguiu aprender as funções AND e OR para diferentes números de entradas (n=2, 3, 5). Isso ocorre porque essas funções são linearmente separáveis. Para n=2, os gráficos do hiperplano mostram claramente a linha de separação encontrada pelo algoritmo. As curvas de aprendizado tipicamente mostram o erro diminuindo até zero, indicando convergência.

**Função XOR:**
Para a função XOR com n=2 entradas, o Perceptron não conseguiu convergir para uma solução com 100% de acurácia. A curva de aprendizado mostra que o número de erros não chega a zero, e o hiperplano de separação não consegue dividir corretamente as classes. Isso demonstra a limitação fundamental do Perceptron: ele só pode resolver problemas linearmente separáveis.
Para XOR com n=3 (paridade), a complexidade é ainda maior e o Perceptron de camada única também falha.

# Exercício 2: Backpropagation (MLP)

### 2.1 Geração de Dados (já definida em 1.1)

In [ ]:
# A função gerar_dados_logicos(n_entradas, operacao) será reutilizada.

### 2.2 Implementação da Rede Neural MLP com Keras

In [ ]:
def criar_modelo_mlp(n_entradas, num_neuronios_oculta=4, func_ativacao_oculta='relu', func_ativacao_saida='sigmoid', learning_rate=0.01, use_bias_option=True):
    model = Sequential()
    # Camada oculta
    # O bias é incluído por padrão (use_bias=True)
    model.add(Dense(num_neuronios_oculta, input_dim=n_entradas, activation=func_ativacao_oculta, use_bias=use_bias_option))
    # Camada de saída
    model.add(Dense(1, activation=func_ativacao_saida, use_bias=use_bias_option))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def plotar_historico_treinamento(history, title):
    plt.figure(figsize=(12, 5))
    
    # Plotar Acurácia
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Acurácia Treino')
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
    plt.title(f'Acurácia: {title}')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.grid(True)

    # Plotar Perda
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Perda Treino')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Perda Validação')
    plt.title(f'Perda: {title}')
    plt.xlabel('Época')
    plt.ylabel('Perda (Loss)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

### 2.3 Testes e Investigações com MLP

In [ ]:
def run_mlp_experiment(n_entradas, operacao, epochs=200, batch_size=4,
                         num_neuronios_oculta=4, func_ativacao_oculta='relu',
                         learning_rate=0.01, use_bias_option=True, verbose=0):
    
    title = f"{operacao} ({n_entradas} entradas), LR={learning_rate}, Act={func_ativacao_oculta}, Bias={use_bias_option}, Neurons={num_neuronios_oculta}"
    print(f"\n--- Testando MLP para {title} ---")
    
    X, y = gerar_dados_logicos(n_entradas, operacao)
    
    # Para problemas pequenos, podemos usar todos os dados para treino e validação
    # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) 
    # Se a base for muito pequena (e.g. 2^2=4 amostras), não faz sentido dividir.
    # Usaremos todos os dados para treino e também para validação para observar.
    X_train, y_train = X, y
    X_val, y_val = X, y

    model = criar_modelo_mlp(n_entradas, num_neuronios_oculta, func_ativacao_oculta, 
                               learning_rate=learning_rate, use_bias_option=use_bias_option)
    
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                          validation_data=(X_val, y_val), verbose=verbose)
    end_time = time.time()
    
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Tempo de treinamento: {end_time - start_time:.4f} segundos")
    print(f"Acurácia Final no conjunto de validação: {accuracy*100:.2f}%")
    print(f"Perda Final: {loss:.4f}")
    
    plotar_historico_treinamento(history, title)
    
    # Mostrar predições para n=2 ou n=3 para verificar
    if n_entradas <= 3:
        print("Predições:")
        preds = model.predict(X, verbose=0)
        for i in range(len(X)):
            print(f"Entrada: {X[i]}, Esperado: {y[i][0]}, Predito: {preds[i][0]:.4f} (Classe: {int(preds[i][0] > 0.5)}) ")
    return model, history

#### 2.3.1 Testes Base (AND, OR, XOR com n=2)

In [ ]:
print("\n=== TESTES BASE MLP (n=2) ===")
run_mlp_experiment(n_entradas=2, operacao='AND', epochs=100, num_neuronios_oculta=4, func_ativacao_oculta='relu', learning_rate=0.05)
run_mlp_experiment(n_entradas=2, operacao='OR', epochs=100, num_neuronios_oculta=4, func_ativacao_oculta='relu', learning_rate=0.05)
run_mlp_experiment(n_entradas=2, operacao='XOR', epochs=300, num_neuronios_oculta=4, func_ativacao_oculta='tanh', learning_rate=0.1) # XOR é mais difícil

#### 2.3.2 Investigação 1: Importância da Taxa de Aprendizado

In [ ]:
print("\n=== INVESTIGAÇÃO: TAXA DE APRENDIZADO (XOR, n=2) ===")
n_xor = 2
op_xor = 'XOR'
neuronios_xor = 4
ativacao_xor = 'tanh'
epocas_xor_lr = 300

taxas_aprendizado = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0]
historicos_lr = {}

for lr in taxas_aprendizado:
    print(f"\nTestando XOR com LR = {lr}")
    _, hist = run_mlp_experiment(n_entradas=n_xor, operacao=op_xor, epochs=epocas_xor_lr, 
                                 num_neuronios_oculta=neuronios_xor, func_ativacao_oculta=ativacao_xor, 
                                 learning_rate=lr, verbose=0)
    historicos_lr[lr] = hist

# Plotar comparação de perdas
plt.figure(figsize=(10, 6))
for lr, hist in historicos_lr.items():
    plt.plot(hist.history['loss'], label=f'LR = {lr}')
plt.title('Comparação de Perda (Loss) para Diferentes Taxas de Aprendizado (XOR n=2)')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)
plt.ylim(0, 1) # Limitar eixo Y para melhor visualização
plt.show()

**Análise da Taxa de Aprendizado:**
Observa-se que:
- Taxas muito baixas (e.g., 0.0001) resultam em convergência excessivamente lenta.
- Taxas muito altas (e.g., 0.5, 1.0) podem levar a instabilidade, com a perda oscilando ou até aumentando, dificultando a convergência.
- Taxas intermediárias (e.g., 0.01, 0.1) tendem a oferecer um bom equilíbrio, permitindo convergência rápida e estável para o problema do XOR.

#### 2.3.3 Investigação 2: Importância do Bias

O bias é incluído por padrão nas camadas `Dense` do Keras (`use_bias=True`). Sua importância é fundamental, pois permite que a função de ativação seja deslocada, aumentando o poder de representação do modelo. Sem bias, a fronteira de decisão da camada é forçada a passar pela origem do espaço de características, o que pode impedir o aprendizado de funções que não têm essa propriedade.

Para investigar, vamos treinar um modelo para XOR com e sem bias na camada oculta e na camada de saída.

In [ ]:
print("\n=== INVESTIGAÇÃO: IMPORTÂNCIA DO BIAS (XOR, n=2) ===")
n_xor_bias = 2
op_xor_bias = 'XOR'
neuronios_xor_bias = 4
ativacao_xor_bias = 'tanh'
lr_xor_bias = 0.1
epocas_xor_bias = 300

print("\n--- Com Bias (Padrão) ---")
_, hist_com_bias = run_mlp_experiment(n_entradas=n_xor_bias, operacao=op_xor_bias, epochs=epocas_xor_bias, 
                                      num_neuronios_oculta=neuronios_xor_bias, func_ativacao_oculta=ativacao_xor_bias, 
                                      learning_rate=lr_xor_bias, use_bias_option=True, verbose=0)

print("\n--- Sem Bias ---")
_, hist_sem_bias = run_mlp_experiment(n_entradas=n_xor_bias, operacao=op_xor_bias, epochs=epocas_xor_bias, 
                                      num_neuronios_oculta=neuronios_xor_bias, func_ativacao_oculta=ativacao_xor_bias, 
                                      learning_rate=lr_xor_bias, use_bias_option=False, verbose=0)

# Plotar comparação de perdas
plt.figure(figsize=(10, 6))
plt.plot(hist_com_bias.history['loss'], label='Com Bias')
plt.plot(hist_sem_bias.history['loss'], label='Sem Bias')
plt.title('Comparação de Perda (Loss) com e sem Bias (XOR n=2)')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)
plt.ylim(0, 1.5) # Ajustar limite se necessário
plt.show()

**Análise da Importância do Bias:**
Como esperado, o modelo treinado sem bias (`use_bias=False`) teve um desempenho significativamente pior, muitas vezes não conseguindo convergir ou atingindo uma acurácia muito baixa para o problema do XOR. Isso ilustra que o bias é crucial para a capacidade da rede de aprender mapeamentos complexos, ajustando o limiar de ativação dos neurônios.

#### 2.3.4 Investigação 3: Importância da Função de Ativação

In [ ]:
print("\n=== INVESTIGAÇÃO: FUNÇÃO DE ATIVAÇÃO (XOR, n=2) ===")
n_xor_act = 2
op_xor_act = 'XOR'
neuronios_xor_act = 4
lr_xor_act = 0.1
epocas_xor_act = 300

funcoes_ativacao = ['sigmoid', 'tanh', 'relu']
historicos_act = {}

for act_func in funcoes_ativacao:
    print(f"\nTestando XOR com Ativação Oculta = {act_func}")
    _, hist = run_mlp_experiment(n_entradas=n_xor_act, operacao=op_xor_act, epochs=epocas_xor_act, 
                                 num_neuronios_oculta=neuronios_xor_act, func_ativacao_oculta=act_func, 
                                 learning_rate=lr_xor_act, verbose=0)
    historicos_act[act_func] = hist

# Plotar comparação de perdas
plt.figure(figsize=(10, 6))
for act_func, hist in historicos_act.items():
    plt.plot(hist.history['loss'], label=f'Ativação = {act_func}')
plt.title('Comparação de Perda (Loss) para Diferentes Funções de Ativação (XOR n=2)')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)
plt.ylim(0, 1) # Ajustar limite
plt.show()

**Análise da Função de Ativação:**
- **Sigmoid:** Consegue aprender o XOR, mas pode ser mais lenta ou propensa a problemas de gradientes evanescentes em redes mais profundas.
- **Tanh:** Frequentemente apresenta melhor desempenho que a sigmoide para camadas ocultas, pois sua saída é centrada em zero, o que pode acelerar a convergência. Para o XOR, geralmente funciona bem.
- **ReLU:** É uma escolha popular devido à sua simplicidade e eficácia em evitar gradientes evanescentes para ativações positivas. Pode levar a convergência rápida, mas é preciso cuidado com o "dying ReLU problem".

Os resultados mostraram que todas as três funções de ativação foram capazes de resolver o XOR, mas `tanh` e `relu` podem oferecer vantagens em termos de velocidade de convergência ou estabilidade, dependendo da configuração.

#### 2.3.5 Testes com Diferentes Números de Entradas ($n$)

In [ ]:
print("\n=== TESTES MLP COM DIFERENTES NÚMEROS DE ENTRADAS ===")

# AND com n=3
run_mlp_experiment(n_entradas=3, operacao='AND', epochs=100, num_neuronios_oculta=6, func_ativacao_oculta='relu', learning_rate=0.05)

# OR com n=4
run_mlp_experiment(n_entradas=4, operacao='OR', epochs=150, num_neuronios_oculta=8, func_ativacao_oculta='relu', learning_rate=0.05)

# XOR com n=3 (Paridade)
# A paridade para n=3 é mais complexa. Pode precisar de mais neurônios/épocas.
run_mlp_experiment(n_entradas=3, operacao='XOR', epochs=500, num_neuronios_oculta=6, func_ativacao_oculta='tanh', learning_rate=0.1)

# XOR com n=4 (Paridade)
# Ainda mais complexo. Aumentar neurônios e épocas.
run_mlp_experiment(n_entradas=4, operacao='XOR', epochs=800, num_neuronios_oculta=8, func_ativacao_oculta='tanh', learning_rate=0.1, batch_size=len(gerar_dados_logicos(4,'XOR')[0])) # batch_size = all data for stability

**Análise para $n$ Entradas:**
As redes MLP foram capazes de aprender as funções AND, OR e XOR (paridade) para um número maior de entradas ($n=3, 4$).
- As funções AND e OR continuam relativamente fáceis de aprender.
- A função XOR (paridade) se torna progressivamente mais difícil à medida que $n$ aumenta. Isso geralmente requer um aumento no número de neurônios na camada oculta (e.g., $2n$ ou mais) e/ou mais épocas de treinamento para alcançar alta acurácia. A escolha da função de ativação (`tanh` ou `relu` costumam ser boas) e da taxa de aprendizado também continua crucial.

### 2.4 Conclusões do Exercício 2

O algoritmo Backpropagation, quando aplicado a Redes Neurais de Múltiplas Camadas (MLP), é uma ferramenta poderosa capaz de resolver problemas de classificação complexos e não linearmente separáveis, como a função XOR com $n$ entradas.

As investigações realizadas destacaram:
1.  **Taxa de Aprendizado:** Sua escolha é crítica. Valores inadequados podem levar à não convergência ou a um treinamento excessivamente lento.
2.  **Bias:** É um componente fundamental para a flexibilidade do modelo. Sem ele, a capacidade da rede de aprender fronteiras de decisão ótimas é severamente limitada.
3.  **Função de Ativação:** A escolha da função de ativação (e.g., Sigmoid, Tanh, ReLU) nas camadas ocultas impacta a dinâmica do treinamento e a eficiência da rede. Tanh e ReLU são geralmente preferidas sobre a Sigmoid para camadas ocultas em muitas aplicações modernas devido à mitigação de problemas como o desaparecimento do gradiente e, no caso do ReLU, eficiência computacional.

A capacidade de generalizar para um número arbitrário de entradas $n$ foi demonstrada, embora a complexidade do problema (especialmente para o XOR generalizado) exija ajustes na arquitetura da rede (número de neurônios, camadas) e nos parâmetros de treinamento.